# 4. 인공신경망
## 4-4강 교차 검증

In [3]:
import pandas as pd # 데이터프레임 형태를 다룰 수 있는 라이브러리
import numpy as np
from sklearn.model_selection import train_test_split # 전체 데이터를 학습 데이터와 평가 데이터로 나눈다.

# ANN
import torch
from torch import nn, optim # torch 내의 세부적인 기능을 불러온다. (신경망 기술, 손실함수, 최적화 방법 등)
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import torch.nn.functional as F # torch 내의 세부적인 기능을 불러온다. (신경망 기술 등)

# Cross Validation
from sklearn.model_selection import KFold

# Plot
import matplotlib.pyplot as plt # 시각화 도구

In [4]:
df = pd.read_csv('./data/reg.csv', index_col=[0])

In [5]:
# 데이터를 넘파이 배열로 만들기
X = df.drop('Price', axis=1).to_numpy() # 데이터프레임에서 타겟값(Price)을 제외하고 넘파이 배열로 만들기
Y = df['Price'].to_numpy().reshape((-1,1)) # 데이터프레임 형태의 타겟값을 넘파이 배열로 만들기

In [6]:
# 텐서 데이터로 변환하는 클래스(3강 참고)
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)
trainset = TensorData(X_train, Y_train)

testset = TensorData(X_test, Y_test)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [8]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(13, 50, bias=True) # 입력층(13) -> 은닉층1(50)으로 가는 연산
        self.fc2 = nn.Linear(50, 30, bias=True) # 은닉층1(50) -> 은닉층2(30)으로 가는 연산
        self.fc3 = nn.Linear(30, 1, bias=True) # 은닉층2(30) -> 출력층(1)으로 가는 연산
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = F.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.  
        x = self.dropout(F.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.(즉, 30개의 20%인 6개의 노드가 계산에서 제외된다.)
        x = F.relu(self.fc3(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다. (가격 예측) 
      
        return x
    
# 주의 사항
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [9]:
kfold = KFold(n_splits=3, shuffle=True)

In [10]:
criterion = nn.MSELoss()

In [11]:
def rmse(dataloader):
        
    with torch.no_grad(): # requires_grad: 비활성화
        square_sum = 0
        num_instances = 0
        model.eval() # 평가를 할 때에는 .eval() 반드시 사용해야 한다.
        for data in dataloader:
            inputs, targets = data
            outputs = model(inputs)
            square_sum += torch.sum((outputs - targets)**2).item()
            num_instances += len(targets)

    model.train()
    return np.sqrt(square_sum/num_instances)  

In [12]:
# 평가 모델들에 대해서 CV 진행 
validation_loss = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
    
    train_subsampler = SubsetRandomSampler(train_idx) # index 생성
    val_subsampler = SubsetRandomSampler(val_idx) # index 생성
    
    # sampler를 이용한 DataLoader 정의
    trainloader = DataLoader(trainset, batch_size=32, sampler=train_subsampler) 
    valloader = DataLoader(trainset, batch_size=32, sampler=val_subsampler)
    
    # 모델
    model = Regressor()
    optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=3e-3)
    
    for epoch in range(200): # 200번 학습을 진행한다.

        for data in trainloader: # 무작위로 섞인 32개 데이터가 있는 배치가 하나 씩 들어온다.

            inputs, values = data # data에는 X, Y가 들어있다.

            optimizer.zero_grad() # 최적화 초기화

            outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
            loss = criterion(outputs, values) # 손실 함수 계산
            loss.backward() # 손실 함수 기준으로 역전파 설정 
            optimizer.step() # 역전파를 진행하고 가중치 업데이트

    train_rmse = rmse(trainloader) # 학습 데이터의 RMSE
    val_rmse = rmse(valloader)
    print("k-fold", fold," Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse)) 
    validation_loss.append(val_rmse)

validation_loss = np.array(validation_loss)
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print("Validation Score: %.4f, ± %.4f" %(mean, std))    

k-fold 0  Train Loss: 0.1188, Validation Loss: 0.1194
k-fold 1  Train Loss: 0.0946, Validation Loss: 0.1575
k-fold 2  Train Loss: 0.1019, Validation Loss: 0.1219
Validation Score: 0.1329, ± 0.0174


In [13]:
# CV를 통해 best model를 정했다면 평가를 진행한다.
# 전체 학습 데이터를 이용한 DataLoader 정의
trainloader = DataLoader(trainset, batch_size=32) 

# 모델
model = Regressor()
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=3e-3)

for epoch in range(200): # 200번 학습을 진행한다.

    for data in trainloader: # 무작위로 섞인 32개 데이터가 있는 배치가 하나 씩 들어온다.

        inputs, values = data # data에는 X, Y가 들어있다.

        optimizer.zero_grad() # 최적화 초기화

        outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
        loss = criterion(outputs, values) # 손실 함수 계산
        loss.backward() # 손실 함수 기준으로 역전파 설정 
        optimizer.step() # 역전파를 진행하고 가중치 업데이트

In [14]:
train_rmse = rmse(trainloader) # 학습 데이터의 RMSE
test_rmse = rmse(testloader) # 시험 데이터의 RMSE
print(" Train Loss: %.4f, Test Loss: %.4f" %(train_rmse, test_rmse)) 

 Train Loss: 0.1021, Test Loss: 0.1108
